# Import Library

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import ast

import warnings
warnings.filterwarnings('ignore')

#visualization libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 10000)

# Read Data

In [2]:
transaction_new = pd.read_csv("../input/fashion-campus/transaction_new.csv")
cust = pd.read_csv("../input/fashion-campus/customer.csv")
products = pd.read_csv('../input/fashion-campus/product.csv', on_bad_lines='skip')
click_new = pd.read_csv('../input/fashion-campus/click_stream_new.csv')

# Labelling

In [3]:
trx2 = transaction_new[transaction_new["payment_status"] == "Success"]
trx2.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753


In [4]:
cust_trx= pd.merge(cust[["customer_id","first_join_date"]],trx2[["customer_id","created_at"]], on = "customer_id", how = "left")
cust_trx

,customer_id,first_join_date,created_at
0,2870,2019-07-21,NaN
1,8193,2017-07-16,NaN
2,7279,2020-08-23,2020-08-23T12:53:12.916342Z
3,7279,2020-08-23,2020-08-23T12:53:12.916342Z
4,7279,2020-08-23,2020-08-31T13:34:59.916342Z
...,...,...,...
1250788,40573,2021-07-17,2021-10-21T15:43:07.139547Z
1250789,40573,2021-07-17,2022-01-23T19:01:48.139547Z
1250790,40573,2021-07-17,2022-03-11T19:03:09.139547Z
1250791,40573,2021-07-17,2022-04-27T20:00:55.139547Z


In [5]:
cust_trx["created_at"] = pd.to_datetime(cust_trx["created_at"])
cust_trx["first_join_date"] = pd.to_datetime(cust_trx["first_join_date"])

In [6]:
cust_trx

,customer_id,first_join_date,created_at
0,2870,2019-07-21,NaT
1,8193,2017-07-16,NaT
2,7279,2020-08-23,2020-08-23 12:53:12.916342+00:00
3,7279,2020-08-23,2020-08-23 12:53:12.916342+00:00
4,7279,2020-08-23,2020-08-31 13:34:59.916342+00:00
...,...,...,...
1250788,40573,2021-07-17,2021-10-21 15:43:07.139547+00:00
1250789,40573,2021-07-17,2022-01-23 19:01:48.139547+00:00
1250790,40573,2021-07-17,2022-03-11 19:03:09.139547+00:00
1250791,40573,2021-07-17,2022-04-27 20:00:55.139547+00:00


In [7]:
cust_trx['created_at'] = cust_trx['created_at'].dt.strftime('%Y-%m')
cust_trx['first_join_date'] = cust_trx['first_join_date'].dt.strftime('%Y-%m')

In [8]:
cust_trx

,customer_id,first_join_date,created_at
0,2870,2019-07,NaN
1,8193,2017-07,NaN
2,7279,2020-08,2020-08
3,7279,2020-08,2020-08
4,7279,2020-08,2020-08
...,...,...,...
1250788,40573,2021-07,2021-10
1250789,40573,2021-07,2022-01
1250790,40573,2021-07,2022-03
1250791,40573,2021-07,2022-04


## Cust nvr trx

In [9]:
cust_nvr_trx = cust_trx[cust_trx["created_at"].isna()]
cust_nvr_trx.shape

(49758, 3)

sebanyak 49.758 (unique) customer tidak pernah transaksi sukses, sisanya yakni 50.242 customer (unique) pernah transaksi sukses

## Cust trx

In [10]:
cust_trx2 = cust_trx[~cust_trx["created_at"].isna()]
cust_trx2.shape

(1201035, 3)

In [11]:
cust_trx2

,customer_id,first_join_date,created_at
2,7279,2020-08,2020-08
3,7279,2020-08,2020-08
4,7279,2020-08,2020-08
5,7279,2020-08,2020-09
6,7279,2020-08,2020-09
...,...,...,...
1250788,40573,2021-07,2021-10
1250789,40573,2021-07,2022-01
1250790,40573,2021-07,2022-03
1250791,40573,2021-07,2022-04


In [12]:
# groupkan berdasarkan customer_id dan created_at
cust_trx4 = cust_trx2.groupby(["customer_id","created_at"])['first_join_date'].first().reset_index()

In [13]:
cust_trx4

,customer_id,created_at,first_join_date
0,3,2018-08,2018-08
1,3,2018-09,2018-08
2,3,2018-10,2018-08
3,3,2018-11,2018-08
4,3,2018-12,2018-08
...,...,...,...
565536,99998,2020-07,2018-07
565537,99998,2021-01,2018-07
565538,99998,2021-07,2018-07
565539,99998,2022-01,2018-07


In [14]:
cust_trx4["created_at"] = pd.to_datetime(cust_trx4["created_at"])

In [15]:
size = cust_trx4.shape[0]
#temp = pd.DataFrame(columns=['A'])
a = []
for i in range (size-1) :
    # jika nilai pada baris ke i di kolom ke 0 == nilai pada baris selanjutnya kolom ke 0 (merupakan cust yg sama)
    if cust_trx4.iloc[i,0] == cust_trx4.iloc[i+1,0] :
        # maka (created_at pada bulan next - created_at pd bulan before) dibagi 1 bulan
        # ini untuk mencari jarak bulan transaksi sukses selanjutnya
        a.append((cust_trx4.iloc[i+1,1] - cust_trx4.iloc[i,1])/np.timedelta64(1,'M'))
    # jika nilai pada baris ke i di kolom ke 0 != nilai pada baris selanjutnya kolom ke 0 (merupakan cust yg berbeda)
    else :
        # maka (created_at pada bulan now - created_at pd bulan now) dibagi 1 bulan
        # alias hasilnya 0, alias itu transaksi terakhir si cust tsb, atau bisa jd dia cuma transaksi sekali
        a.append((cust_trx4.iloc[i,1] - cust_trx4.iloc[i,1])/np.timedelta64(1,'M'))

In [16]:
diff = pd.DataFrame(a, columns = ["diff"])
diff = round(diff)
diff.head()

,diff
0,1.0
1,1.0
2,1.0
3,1.0
4,2.0


In [17]:
# langsung aja concat kalo emang jumlah rowsnya sama dan yakin dia emang seharusnya ada di row itu

cust_trx5 = pd.concat([cust_trx4, diff], axis=1)
cust_trx5["diff"] = cust_trx5["diff"].fillna(0)
cust_trx5

,customer_id,created_at,first_join_date,diff
0,3,2018-08-01,2018-08,1.0
1,3,2018-09-01,2018-08,1.0
2,3,2018-10-01,2018-08,1.0
3,3,2018-11-01,2018-08,1.0
4,3,2018-12-01,2018-08,2.0
...,...,...,...,...
565536,99998,2020-07-01,2018-07,6.0
565537,99998,2021-01-01,2018-07,6.0
565538,99998,2021-07-01,2018-07,6.0
565539,99998,2022-01-01,2018-07,0.0


disini, 1 berati jarak antara transaksi bulan sekarang dengan transaksi selanjutnya, berjarak 1 bulan. kalo 0 berati dah gada transaksi lagi (ga dipake), kalo 2,3dst berati ya jarak antara transaksi bulan ini dgn transaksi selanjutnya berjarak sesuai angka itu (churn)

In [18]:
def renovated(df) : 
    if df["diff"] == 1 : 
        return 0
    elif df["diff"] == 0 :
        return 2
    else :
        return 1

    
cust_trx5["flagging"] = cust_trx5.apply(renovated, axis =1 )

disni berati, ketika jarak transaksi sekarang dgn selanjutnya adalah 1 bulan, dia akan dilabeli 0, maka dia terhitung transaksi rutin. kalo jaraknya 0 bulan, dia akan dilabeli 2, dan ketika jarak ke transaksi selanjutnya adalah di atas 1 bulan, dia akan dilabeli 1 (churn)

In [19]:
cust_trx5

,customer_id,created_at,first_join_date,diff,flagging
0,3,2018-08-01,2018-08,1.0,0
1,3,2018-09-01,2018-08,1.0,0
2,3,2018-10-01,2018-08,1.0,0
3,3,2018-11-01,2018-08,1.0,0
4,3,2018-12-01,2018-08,2.0,1
...,...,...,...,...,...
565536,99998,2020-07-01,2018-07,6.0,1
565537,99998,2021-01-01,2018-07,6.0,1
565538,99998,2021-07-01,2018-07,6.0,1
565539,99998,2022-01-01,2018-07,0.0,2


In [20]:
final_flagging = cust_trx5[cust_trx5["flagging"] <= 1].drop(columns=['diff'])
final_flagging

,customer_id,created_at,first_join_date,flagging
0,3,2018-08-01,2018-08,0
1,3,2018-09-01,2018-08,0
2,3,2018-10-01,2018-08,0
3,3,2018-11-01,2018-08,0
4,3,2018-12-01,2018-08,1
...,...,...,...,...
565534,99998,2019-07-01,2018-07,1
565535,99998,2020-01-01,2018-07,1
565536,99998,2020-07-01,2018-07,1
565537,99998,2021-01-01,2018-07,1


nah disini, yang label 2 di drop, yang mana jika cust tsb cuma trans sekali selama ini atau itu adalah transaksi terakhirnya, maka dia tidak akan diikutkan ke dalam pembuatan untuk feature

In [21]:
final_flagging["flagging"].value_counts(normalize = True)

0    0.620106
1    0.379894
Name: flagging, dtype: float64

- 1 adalah nasabah churn
- 0 adalah nasabah yang trx rutin

# Feature Engineering

## 1. Traffic Source

In [22]:
# ini karena masing2 device pasti punya sessionnya masing-masing. tidak mungkin device yang berbeda
# memiliki session_id yang sama

click_traffic = click_new.groupby(['session_id'])[['traffic_source']].first().reset_index()

In [23]:
click_traffic

,session_id,traffic_source
0,00001f29-3ede-4445-95f0-0178934e32ba,MOBILE
1,00003eca-954b-4150-aee1-63fc62f395cf,MOBILE
2,0000425b-ddc8-4459-a240-6ba814dac132,MOBILE
3,00005746-17ec-449c-9c5c-699a3ea9304a,MOBILE
4,0000649b-9514-4600-979b-42166a798996,MOBILE
...,...,...
895198,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,WEB
895199,ffffd3d9-79db-459e-a427-388204a7aac7,MOBILE
895200,ffffdf8e-dea1-42bc-9839-7662675c0d38,MOBILE
895201,fffff7cf-2a18-4303-9c12-aa836ae3b543,MOBILE


In [24]:
trx_click = pd.merge(trx2[['created_at','customer_id','session_id']], click_traffic[['session_id','traffic_source']], on='session_id')

trx_click['created_at'] = pd.to_datetime(trx_click["created_at"])
trx_click['created_at'] = trx_click['created_at'].dt.strftime('%Y-%m')
trx_click['created_at'] = pd.to_datetime(trx_click["created_at"])

trx_click2 = trx_click.groupby(['customer_id','created_at','traffic_source']).count().reset_index()
trx_click2

,customer_id,created_at,traffic_source,session_id
0,3,2018-08-01,MOBILE,1
1,3,2018-09-01,MOBILE,1
2,3,2018-10-01,MOBILE,1
3,3,2018-11-01,MOBILE,1
4,3,2018-12-01,MOBILE,1
...,...,...,...,...
599993,99998,2020-07-01,MOBILE,2
599994,99998,2021-01-01,MOBILE,1
599995,99998,2021-07-01,MOBILE,1
599996,99998,2022-01-01,MOBILE,16


In [25]:
# trx_click2[trx_click2[['customer_id','created_at']].duplicated() == True]
trx_click2[trx_click2['customer_id'] == 99995]

,customer_id,created_at,traffic_source,session_id
599962,99995,2020-08-01,MOBILE,2
599963,99995,2020-09-01,MOBILE,6
599964,99995,2020-10-01,MOBILE,5
599965,99995,2020-11-01,MOBILE,2
599966,99995,2020-12-01,MOBILE,3
599967,99995,2021-01-01,MOBILE,2
599968,99995,2021-02-01,MOBILE,4
599969,99995,2021-03-01,MOBILE,3
599970,99995,2021-04-01,MOBILE,3
599971,99995,2021-05-01,MOBILE,3


In [26]:
# one hot encoding
trx_click3 = pd.get_dummies(trx_click2['traffic_source'])
trx_click3 = trx_click3.rename(columns = {
    'MOBILE' : 'mobile_used',
    'WEB' : 'web_used'
})
trx_click3

,mobile_used,web_used
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
599993,1,0
599994,1,0
599995,1,0
599996,1,0


In [27]:
# trx_click3 = trx_click2.pivot_table('session_id', ['customer_id','created_at'], 'traffic_source', aggfunc=len, fill_value=0).reset_index()
# trx_click3 = trx_click3.rename(columns = {
#     'MOBILE' : 'mobile_used',
#     'WEB' : 'web_used'
# })
trx_click4 = pd.concat([trx_click2, trx_click3], axis=1)
trx_click5 = trx_click4.groupby(['customer_id','created_at']).sum().reset_index()

trx_click5

,customer_id,created_at,session_id,mobile_used,web_used
0,3,2018-08-01,1,1,0
1,3,2018-09-01,1,1,0
2,3,2018-10-01,1,1,0
3,3,2018-11-01,1,1,0
4,3,2018-12-01,1,1,0
...,...,...,...,...,...
565534,99998,2020-07-01,2,1,0
565535,99998,2021-01-01,1,1,0
565536,99998,2021-07-01,1,1,0
565537,99998,2022-01-01,16,1,0


In [28]:
# trx_click5[trx_click5['mobile_used'] > 1]
trx_click5[trx_click5['web_used'] > 1]

,customer_id,created_at,session_id,mobile_used,web_used


In [29]:
# trx_click4 = final_flagging.merge(trx_click4, on=['customer_id','created_at'])
# trx_click4

trx_click6 = final_flagging.merge(trx_click5.iloc[:,[0,1,3,4]], on=['customer_id','created_at'], how='left')
trx_click6

,customer_id,created_at,first_join_date,flagging,mobile_used,web_used
0,3,2018-08-01,2018-08,0,1.0,0.0
1,3,2018-09-01,2018-08,0,1.0,0.0
2,3,2018-10-01,2018-08,0,1.0,0.0
3,3,2018-11-01,2018-08,0,1.0,0.0
4,3,2018-12-01,2018-08,1,1.0,0.0
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,1.0,0.0
515295,99998,2020-01-01,2018-07,1,1.0,0.0
515296,99998,2020-07-01,2018-07,1,1.0,0.0
515297,99998,2021-01-01,2018-07,1,1.0,0.0


In [30]:
asess_1 = pd.crosstab(trx_click6["web_used"], trx_click6["flagging"], margins=True, margins_name="Total")
asess_1

flagging,0,1,Total
web_used,,,
0.0,267957,176132,444089
1.0,51583,19626,71209
Total,319540,195758,515298


In [31]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195758)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_1["%nonevent"] = asess_1.apply(nonevent, axis = 1)
asess_1["%event"] = asess_1.apply(event, axis = 1)

In [32]:
#Let's calculate WoE and save it to new a column WoE
asess_1["WoE"] = np.log(asess_1["%event"]/asess_1["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_1["IV"] = asess_1["WoE"]*(asess_1["%event"] - asess_1["%nonevent"])

#Let's sum IV
asess_1["IV"].sum() # Weak Predictor

0.03344570026681302

## 2. Quantity Item Add_to_Cart

In [33]:
click_add = click_new[click_new['event_name'] == 'ADD_TO_CART'][['session_id','quantity']].groupby('session_id').sum().reset_index()
click_add

,session_id,quantity
0,00001f29-3ede-4445-95f0-0178934e32ba,6.0
1,00003eca-954b-4150-aee1-63fc62f395cf,1.0
2,0000425b-ddc8-4459-a240-6ba814dac132,1.0
3,00005746-17ec-449c-9c5c-699a3ea9304a,6.0
4,0000649b-9514-4600-979b-42166a798996,3.0
...,...,...
872056,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,2.0
872057,ffffd3d9-79db-459e-a427-388204a7aac7,18.0
872058,ffffdf8e-dea1-42bc-9839-7662675c0d38,1.0
872059,fffff7cf-2a18-4303-9c12-aa836ae3b543,5.0


In [34]:
trx_add = pd.merge(trx2[['created_at','customer_id','session_id']], click_add, on='session_id')

trx_add['created_at'] = pd.to_datetime(trx_add["created_at"])
trx_add['created_at'] = trx_add['created_at'].dt.strftime('%Y-%m')
trx_add['created_at'] = pd.to_datetime(trx_add["created_at"])

trx_add2 = trx_add.groupby(['customer_id','created_at'])['quantity'].sum().reset_index()

In [35]:
trx_add2.describe()

,customer_id,quantity
count,565539.000000,565539.000000
mean,49868.677969,12.250031
std,28969.808969,47.946768
min,3.000000,1.000000
25%,24741.000000,1.000000
50%,49624.000000,3.000000
75%,74948.000000,8.000000
max,99999.000000,4117.000000


In [36]:
trx_add2[trx_add2['customer_id'] == 3]

,customer_id,created_at,quantity
0,3,2018-08-01,6.0
1,3,2018-09-01,3.0
2,3,2018-10-01,3.0
3,3,2018-11-01,3.0
4,3,2018-12-01,6.0
5,3,2019-02-01,1.0
6,3,2019-03-01,4.0
7,3,2019-04-01,2.0
8,3,2019-05-01,6.0
9,3,2019-06-01,13.0


In [37]:
trx_add3 = final_flagging.merge(trx_add2, on=['customer_id','created_at'], how='left')
trx_add3 = trx_add3.fillna(0)
trx_add3

,customer_id,created_at,first_join_date,flagging,quantity
0,3,2018-08-01,2018-08,0,6.0
1,3,2018-09-01,2018-08,0,3.0
2,3,2018-10-01,2018-08,0,3.0
3,3,2018-11-01,2018-08,0,3.0
4,3,2018-12-01,2018-08,1,6.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,2.0
515295,99998,2020-01-01,2018-07,1,1.0
515296,99998,2020-07-01,2018-07,1,6.0
515297,99998,2021-01-01,2018-07,1,1.0


In [38]:
trx_add3.describe()

,customer_id,flagging,quantity
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,12.330862
std,28972.061729,0.485361,48.238974
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,1.000000
50%,49619.000000,0.000000,3.000000
75%,74951.000000,1.000000,8.000000
max,99998.000000,1.000000,4117.000000


In [39]:
def func(x):
    if x <= 2:
        return 1 #melakukan binning
    elif x <= 8:
        return 2
    else:
        return 3

trx_add3['add_to_cart_item'] = trx_add3['quantity'].apply(func)

In [40]:
trx_add3

,customer_id,created_at,first_join_date,flagging,quantity,add_to_cart_item
0,3,2018-08-01,2018-08,0,6.0,2
1,3,2018-09-01,2018-08,0,3.0,2
2,3,2018-10-01,2018-08,0,3.0,2
3,3,2018-11-01,2018-08,0,3.0,2
4,3,2018-12-01,2018-08,1,6.0,2
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,2.0,1
515295,99998,2020-01-01,2018-07,1,1.0,1
515296,99998,2020-07-01,2018-07,1,6.0,2
515297,99998,2021-01-01,2018-07,1,1.0,1


In [41]:
# trx_add3 = trx_add2.pivot_table('quantity', ['customer_id','created_at'], 'add_to_cart_item', aggfunc=sum, fill_value=0).reset_index()
# trx_add3['few_item_add'] = trx_add3['few_item_add'].apply(lambda x : 0 if x == 0 else 1)
# trx_add3['many_item_add'] = trx_add3['many_item_add'].apply(lambda x : 0 if x == 0 else 1)
# trx_add3['normal_item_add'] = trx_add3['normal_item_add'].apply(lambda x : 0 if x == 0 else 1)

In [42]:
asess_2 = pd.crosstab(trx_add3["add_to_cart_item"], trx_add3["flagging"], margins=True, margins_name="Total")
asess_2

flagging,0,1,Total
add_to_cart_item,,,
1,139719,89779,229498
2,98784,70018,168802
3,81037,35962,116999
Total,319540,195759,515299


In [43]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_2["%nonevent"] = asess_2.apply(nonevent, axis = 1)
asess_2["%event"] = asess_2.apply(event, axis = 1)

In [44]:
#Let's calculate WoE and save it to new a column WoE
asess_2["WoE"] = np.log(asess_2["%event"]/asess_2["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_2["IV"] = asess_2["WoE"]*(asess_2["%event"] - asess_2["%nonevent"])

#Let's sum IV
asess_2["IV"].sum() # Weak Predictor

0.030634882381357215

## 3. Promo amount

In [45]:
# ketika booking_id sama, dan pake promo yang sama, maka penggunaan promo dihitung 1 (harus groupby booking_id
# dan aggregate promo_amount first)
# baru setelah itu digroupby sum berbulan (created_at)

trx_promo = trx2[['created_at','customer_id','booking_id','promo_amount']].copy()

trx_promo['created_at'] = pd.to_datetime(trx_promo["created_at"])
trx_promo['created_at'] = trx_promo['created_at'].dt.strftime('%Y-%m')
trx_promo['created_at'] = pd.to_datetime(trx_promo["created_at"])

trx_promo2_first = trx_promo.groupby(['customer_id','created_at','booking_id'])[['promo_amount']].first().reset_index()

In [46]:
trx_promo2_first

,customer_id,created_at,booking_id,promo_amount
0,3,2018-08-01,856b0c3d-b076-406b-be11-3dfee762e9be,8073
1,3,2018-09-01,f9b17de8-5db3-40dc-9227-664c6679543c,0
2,3,2018-10-01,caa8be57-6239-4b5b-ae0b-e27887a13356,8828
3,3,2018-11-01,688cef52-8ee7-44ea-98af-20daf0c5609b,3415
4,3,2018-12-01,a12fb778-d19a-4559-9808-8115f01c1595,2645
...,...,...,...,...
815959,99998,2020-07-01,0d102b41-78e1-4c46-81d2-eff6f725fc2a,0
815960,99998,2021-01-01,d7a09532-f774-4528-9d6e-be9ef2d67bef,8569
815961,99998,2021-07-01,0361a084-9362-4a93-b940-d59e7ce91970,0
815962,99998,2022-01-01,06fbe4b4-e98b-403d-a05d-396ae9903e2a,1954


In [47]:
trx_promo2 = trx_promo2_first.groupby(['customer_id','created_at'])[['promo_amount']].sum().reset_index()
trx_promo2

,customer_id,created_at,promo_amount
0,3,2018-08-01,8073
1,3,2018-09-01,0
2,3,2018-10-01,8828
3,3,2018-11-01,3415
4,3,2018-12-01,2645
...,...,...,...
565536,99998,2020-07-01,0
565537,99998,2021-01-01,8569
565538,99998,2021-07-01,0
565539,99998,2022-01-01,1954


In [48]:
trx_promo3 = final_flagging.merge(trx_promo2, on=['customer_id','created_at'], how='left')
trx_promo3 = trx_promo3.fillna(0)
trx_promo3

,customer_id,created_at,first_join_date,flagging,promo_amount
0,3,2018-08-01,2018-08,0,8073
1,3,2018-09-01,2018-08,0,0
2,3,2018-10-01,2018-08,0,8828
3,3,2018-11-01,2018-08,0,3415
4,3,2018-12-01,2018-08,1,2645
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,5670
515295,99998,2020-01-01,2018-07,1,4912
515296,99998,2020-07-01,2018-07,1,0
515297,99998,2021-01-01,2018-07,1,8569


In [49]:
trx_promo3.describe()

,customer_id,flagging,promo_amount
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,3004.977757
std,28972.061729,0.485361,4163.880206
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,0.000000
50%,49619.000000,0.000000,0.000000
75%,74951.000000,1.000000,5150.000000
max,99998.000000,1.000000,70409.000000


In [50]:
def func(x):
    if x == 0:
        return 1 # melakukan binning
    elif x <= 5500:
        return 2
    else:
        return 3


trx_promo3['promo_amount_used'] = trx_promo3['promo_amount'].apply(func)

In [51]:
trx_promo3

,customer_id,created_at,first_join_date,flagging,promo_amount,promo_amount_used
0,3,2018-08-01,2018-08,0,8073,3
1,3,2018-09-01,2018-08,0,0,1
2,3,2018-10-01,2018-08,0,8828,3
3,3,2018-11-01,2018-08,0,3415,2
4,3,2018-12-01,2018-08,1,2645,2
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,5670,3
515295,99998,2020-01-01,2018-07,1,4912,2
515296,99998,2020-07-01,2018-07,1,0,1
515297,99998,2021-01-01,2018-07,1,8569,3


In [52]:
asess_3 = pd.crosstab(trx_promo3["promo_amount_used"], trx_promo3["flagging"], margins=True, margins_name="Total")
asess_3

flagging,0,1,Total
promo_amount_used,,,
1,154835,115058,269893
2,79408,48665,128073
3,85297,32036,117333
Total,319540,195759,515299


In [53]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_3["%nonevent"] = asess_3.apply(nonevent, axis = 1)
asess_3["%event"] = asess_3.apply(event, axis = 1)

In [54]:
#Let's calculate WoE and save it to new a column WoE
asess_3["WoE"] = np.log(asess_3["%event"]/asess_3["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_3["IV"] = asess_3["WoE"]*(asess_3["%event"] - asess_3["%nonevent"])

#Let's sum IV
asess_3["IV"].sum() # Weak Predictor

0.07046099037727191

## 4. Payment method

In [55]:
trx_method = trx2[['created_at','customer_id','payment_method']].copy()

trx_method['created_at'] = pd.to_datetime(trx_method["created_at"])
trx_method['created_at'] = trx_method['created_at'].dt.strftime('%Y-%m')
trx_method['created_at'] = pd.to_datetime(trx_method["created_at"])

trx_method2 = trx_method.groupby(['customer_id','created_at','payment_method']).count().reset_index()
trx_method2

,customer_id,created_at,payment_method
0,3,2018-08-01,Credit Card
1,3,2018-09-01,Credit Card
2,3,2018-10-01,Credit Card
3,3,2018-11-01,Credit Card
4,3,2018-12-01,Credit Card
...,...,...,...
636100,99998,2020-07-01,Credit Card
636101,99998,2021-01-01,Credit Card
636102,99998,2021-07-01,Debit Card
636103,99998,2022-01-01,Credit Card


In [56]:
trx_method2[trx_method2[['customer_id','created_at']].duplicated() == True]

,customer_id,created_at,payment_method
21,3,2020-05-01,LinkAja
34,3,2021-05-01,Gopay
46,3,2022-05-01,Debit Card
92,20,2021-11-01,OVO
94,20,2021-12-01,Debit Card
...,...,...,...
636077,99995,2021-06-01,Debit Card
636083,99995,2021-11-01,Debit Card
636085,99995,2021-12-01,Gopay
636089,99995,2022-03-01,OVO


In [57]:
# one hot encoding
trx_method3 = pd.get_dummies(trx_method2['payment_method'])
trx_method3 = trx_method3.rename(columns = {
    'Debit Card' : 'debit_card',
    'Credit Card' : 'credit_card',
    'Gopay' : 'gopay',
    'LinkAja' : 'linkAja',
    'OVO' : 'ovo'
})
trx_method3

,credit_card,debit_card,gopay,linkAja,ovo
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
636100,1,0,0,0,0
636101,1,0,0,0,0
636102,0,1,0,0,0
636103,1,0,0,0,0


In [58]:
trx_method4 = pd.concat([trx_method2, trx_method3], axis=1)
trx_method5 = trx_method4.groupby(['customer_id','created_at']).sum().reset_index()

trx_method5

,customer_id,created_at,credit_card,debit_card,gopay,linkAja,ovo
0,3,2018-08-01,1,0,0,0,0
1,3,2018-09-01,1,0,0,0,0
2,3,2018-10-01,1,0,0,0,0
3,3,2018-11-01,1,0,0,0,0
4,3,2018-12-01,1,0,0,0,0
...,...,...,...,...,...,...,...
565536,99998,2020-07-01,1,0,0,0,0
565537,99998,2021-01-01,1,0,0,0,0
565538,99998,2021-07-01,0,1,0,0,0
565539,99998,2022-01-01,1,0,0,0,0


In [59]:
trx_method6 = final_flagging.merge(trx_method5, on=['customer_id','created_at'], how='left')
trx_method6

,customer_id,created_at,first_join_date,flagging,credit_card,debit_card,gopay,linkAja,ovo
0,3,2018-08-01,2018-08,0,1,0,0,0,0
1,3,2018-09-01,2018-08,0,1,0,0,0,0
2,3,2018-10-01,2018-08,0,1,0,0,0,0
3,3,2018-11-01,2018-08,0,1,0,0,0,0
4,3,2018-12-01,2018-08,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,1,0,0,0,0
515295,99998,2020-01-01,2018-07,1,0,1,0,0,0
515296,99998,2020-07-01,2018-07,1,1,0,0,0,0
515297,99998,2021-01-01,2018-07,1,1,0,0,0,0


## 5. shipment fee

In [60]:
trx_shipment = trx2[['created_at','customer_id','shipment_fee']].copy()

trx_shipment['created_at'] = pd.to_datetime(trx_shipment["created_at"])
trx_shipment['created_at'] = trx_shipment['created_at'].dt.strftime('%Y-%m')
trx_shipment['created_at'] = pd.to_datetime(trx_shipment["created_at"])

trx_shipment2 = trx_shipment.groupby(['customer_id','created_at']).sum().reset_index()
trx_shipment2

,customer_id,created_at,shipment_fee
0,3,2018-08-01,0
1,3,2018-09-01,10000
2,3,2018-10-01,10000
3,3,2018-11-01,0
4,3,2018-12-01,10000
...,...,...,...
565536,99998,2020-07-01,20000
565537,99998,2021-01-01,10000
565538,99998,2021-07-01,0
565539,99998,2022-01-01,160000


In [61]:
trx_shipment3 = final_flagging.merge(trx_shipment2, on=['customer_id','created_at'], how='left')
trx_shipment3

,customer_id,created_at,first_join_date,flagging,shipment_fee
0,3,2018-08-01,2018-08,0,0
1,3,2018-09-01,2018-08,0,10000
2,3,2018-10-01,2018-08,0,10000
3,3,2018-11-01,2018-08,0,0
4,3,2018-12-01,2018-08,1,10000
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,10000
515295,99998,2020-01-01,2018-07,1,0
515296,99998,2020-07-01,2018-07,1,20000
515297,99998,2021-01-01,2018-07,1,10000


In [62]:
trx_shipment3.describe()

,customer_id,flagging,shipment_fee
count,515299.000000,515299.000000,5.152990e+05
mean,49873.821358,0.379894,1.970034e+04
std,28972.061729,0.485361,3.275937e+04
min,3.000000,0.000000,0.000000e+00
25%,24742.000000,0.000000,1.000000e+04
50%,49619.000000,0.000000,1.000000e+04
75%,74951.000000,1.000000,2.000000e+04
max,99998.000000,1.000000,1.640000e+06


In [63]:
def func(x):
    if x <= 10000:
        return 1 # melakukan binning
    elif x <= 20000:
        return 2
    else:
        return 3


trx_shipment3['shipments_fee'] = trx_shipment3['shipment_fee'].apply(func)

In [64]:
asess_5 = pd.crosstab(trx_shipment3["shipments_fee"], trx_shipment3["flagging"], margins=True, margins_name="Total")
asess_5

flagging,0,1,Total
shipments_fee,,,
1,172467,150255,322722
2,54078,20040,74118
3,92995,25464,118459
Total,319540,195759,515299


In [65]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_5["%nonevent"] = asess_5.apply(nonevent, axis = 1)
asess_5["%event"] = asess_5.apply(event, axis = 1)

In [66]:
#Let's calculate WoE and save it to new a column WoE
asess_5["WoE"] = np.log(asess_5["%event"]/asess_5["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_5["IV"] = asess_5["WoE"]*(asess_5["%event"] - asess_5["%nonevent"])

#Let's sum IV
asess_5["IV"].sum() # Medium Predictor

0.24344309319640933

In [67]:
trx_shipment3

,customer_id,created_at,first_join_date,flagging,shipment_fee,shipments_fee
0,3,2018-08-01,2018-08,0,0,1
1,3,2018-09-01,2018-08,0,10000,1
2,3,2018-10-01,2018-08,0,10000,1
3,3,2018-11-01,2018-08,0,0,1
4,3,2018-12-01,2018-08,1,10000,1
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,10000,1
515295,99998,2020-01-01,2018-07,1,0,1
515296,99998,2020-07-01,2018-07,1,20000,2
515297,99998,2021-01-01,2018-07,1,10000,1


## 6. shipment date limit

In [68]:
trx_datelimit = trx2[['created_at','customer_id','shipment_date_limit']].copy()

trx_datelimit['created_at'] = pd.to_datetime(trx_datelimit["created_at"])
trx_datelimit['shipment_date_limit'] = pd.to_datetime(trx_datelimit["shipment_date_limit"])

trx_datelimit['diff_datelimit'] = trx_datelimit['shipment_date_limit'] - trx_datelimit['created_at']

trx_datelimit['created_at'] = trx_datelimit['created_at'].dt.strftime('%Y-%m')
trx_datelimit['created_at'] = pd.to_datetime(trx_datelimit["created_at"])

trx_datelimit2 = trx_datelimit.groupby(['customer_id','created_at'])[['diff_datelimit']].mean().reset_index()
trx_datelimit2

,customer_id,created_at,diff_datelimit
0,3,2018-08-01,1 days 10:14:11.973051
1,3,2018-09-01,3 days 00:32:02.888480
2,3,2018-10-01,5 days 10:19:03.419877
3,3,2018-11-01,3 days 09:56:18.435177
4,3,2018-12-01,3 days 01:42:02.245432
...,...,...,...
565536,99998,2020-07-01,4 days 19:30:47.205880
565537,99998,2021-01-01,1 days 00:52:13.840735
565538,99998,2021-07-01,2 days 03:36:13.194951
565539,99998,2022-01-01,1 days 17:13:40.615007


In [69]:
trx_datelimit2.describe()

,customer_id,diff_datelimit
count,565541.000000,565541
mean,49868.684955,3 days 12:26:44.867104319
std,28969.757982,1 days 08:12:25.663020245
min,3.000000,1 days 00:00:02.200453
25%,24741.000000,2 days 10:29:17.947286400
50%,49624.000000,3 days 12:28:55.253620
75%,74948.000000,4 days 14:21:41.798299
max,99999.000000,6 days 00:59:59.332085


In [70]:
from datetime import datetime, timedelta

# menentukan kategori
def func(row):
    if row <= timedelta(days = 2):
        return 1
    elif row <= timedelta(days = 4):
        return 2
    elif row <= timedelta(days = 5):
        return 3
    else:
        return 4

trx_datelimit2['diff_datelimits'] = trx_datelimit2['diff_datelimit'].apply(func)

In [71]:
trx_datelimit3 = final_flagging.merge(trx_datelimit2, on=['customer_id','created_at'])
trx_datelimit3

,customer_id,created_at,first_join_date,flagging,diff_datelimit,diff_datelimits
0,3,2018-08-01,2018-08,0,1 days 10:14:11.973051,1
1,3,2018-09-01,2018-08,0,3 days 00:32:02.888480,2
2,3,2018-10-01,2018-08,0,5 days 10:19:03.419877,4
3,3,2018-11-01,2018-08,0,3 days 09:56:18.435177,2
4,3,2018-12-01,2018-08,1,3 days 01:42:02.245432,2
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,3 days 01:25:27.973391,2
515295,99998,2020-01-01,2018-07,1,2 days 05:33:26.784554,2
515296,99998,2020-07-01,2018-07,1,4 days 19:30:47.205880,3
515297,99998,2021-01-01,2018-07,1,1 days 00:52:13.840735,1


In [72]:
asess_6 = pd.crosstab(trx_datelimit3["diff_datelimits"], trx_datelimit3["flagging"], margins=True, margins_name="Total")
asess_6

flagging,0,1,Total
diff_datelimits,,,
1,44036,37578,81614
2,156875,78612,235487
3,71078,39307,110385
4,47551,40262,87813
Total,319540,195759,515299


In [73]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_6["%nonevent"] = asess_6.apply(nonevent, axis = 1)
asess_6["%event"] = asess_6.apply(event, axis = 1)


In [74]:
#Let's calculate WoE and save it to new a column WoE
asess_6["WoE"] = np.log(asess_6["%event"]/asess_6["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_6["IV"] = asess_6["WoE"]*(asess_6["%event"] - asess_6["%nonevent"])

#Let's sum IV
asess_6["IV"].sum() # Weak Predictor

0.05651782273549739

## 7. click item detail 

In [75]:
click_new['event_name'].value_counts()

CLICK          2498038
HOMEPAGE       2487126
ADD_TO_CART    1937157
SCROLL         1663571
ITEM_DETAIL    1290396
SEARCH         1173266
BOOKING         852582
PROMO_PAGE      604931
ADD_PROMO       326535
Name: event_name, dtype: int64

In [76]:
click_item = click_new[click_new['event_name'] == 'ITEM_DETAIL'][['session_id']]
click_item2 = trx2.iloc[:,[0,1,3]].merge(click_item, on='session_id')

click_item2['created_at'] = pd.to_datetime(click_item2["created_at"])
click_item2['created_at'] = click_item2['created_at'].dt.strftime('%Y-%m')
click_item2['created_at'] = pd.to_datetime(click_item2["created_at"])

In [77]:
click_item3 = click_item2.groupby(['customer_id','created_at']).count().reset_index()
click_item3 = click_item3.rename(columns={
    'session_id' : 'click_detail_item'
})
click_item3

,customer_id,created_at,click_detail_item
0,3,2018-10-01,2
1,3,2018-11-01,2
2,3,2018-12-01,1
3,3,2019-02-01,1
4,3,2019-03-01,2
...,...,...,...
363257,99998,2019-07-01,27
363258,99998,2021-01-01,2
363259,99998,2021-07-01,2
363260,99998,2022-01-01,32


In [78]:
click_item4 = final_flagging.merge(click_item3, on=['customer_id','created_at'], how='left')
click_item4 = click_item4.fillna(0)
click_item4

,customer_id,created_at,first_join_date,flagging,click_detail_item
0,3,2018-08-01,2018-08,0,0.0
1,3,2018-09-01,2018-08,0,0.0
2,3,2018-10-01,2018-08,0,2.0
3,3,2018-11-01,2018-08,0,2.0
4,3,2018-12-01,2018-08,1,1.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,27.0
515295,99998,2020-01-01,2018-07,1,0.0
515296,99998,2020-07-01,2018-07,1,0.0
515297,99998,2021-01-01,2018-07,1,2.0


In [79]:
click_item4.describe()

,customer_id,flagging,click_detail_item
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,3.282605
std,28972.061729,0.485361,7.314569
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,0.000000
50%,49619.000000,0.000000,1.000000
75%,74951.000000,1.000000,4.000000
max,99998.000000,1.000000,420.000000


In [80]:
# menentukan kategori
def func(row):
    if row <= 1:
        return 1
    elif row <= 4:
        return 2
    else:
        return 3

click_item4['click_detail_items'] = click_item4['click_detail_item'].apply(func)

In [81]:
asess_7 = pd.crosstab(click_item4["click_detail_items"], click_item4["flagging"], margins=True, margins_name="Total")
asess_7

flagging,0,1,Total
click_detail_items,,,
1,145788,136455,282243
2,89471,43268,132739
3,84281,16036,100317
Total,319540,195759,515299


In [82]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_7["%nonevent"] = asess_7.apply(nonevent, axis = 1)
asess_7["%event"] = asess_7.apply(event, axis = 1)

In [83]:
#Let's calculate WoE and save it to new a column WoE
asess_7["WoE"] = np.log(asess_7["%event"]/asess_7["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_7["IV"] = asess_7["WoE"]*(asess_7["%event"] - asess_7["%nonevent"])

#Let's sum IV
asess_7["IV"].sum() # strong Predictor

0.32864292394681305

## 8. click homepage

In [84]:
click_home = click_new[click_new['event_name'] == 'HOMEPAGE'][['session_id']]
click_home2 = trx2.iloc[:,[0,1,3]].merge(click_home, on='session_id')

click_home2['created_at'] = pd.to_datetime(click_home2["created_at"])
click_home2['created_at'] = click_home2['created_at'].dt.strftime('%Y-%m')
click_home2['created_at'] = pd.to_datetime(click_home2["created_at"])

In [85]:
click_home3 = click_home2.groupby(['customer_id','created_at']).count().reset_index()
click_home3 = click_home3.rename(columns={
    'session_id' : 'click_homepage'
})
click_home3

,customer_id,created_at,click_homepage
0,3,2018-08-01,1
1,3,2018-09-01,1
2,3,2018-10-01,4
3,3,2018-11-01,1
4,3,2018-12-01,3
...,...,...,...
565534,99998,2020-07-01,2
565535,99998,2021-01-01,3
565536,99998,2021-07-01,2
565537,99998,2022-01-01,16


In [86]:
click_home4 = final_flagging.merge(click_home3, on=['customer_id','created_at'], how='left')
click_home4 = click_home4.fillna(0)
click_home4

,customer_id,created_at,first_join_date,flagging,click_homepage
0,3,2018-08-01,2018-08,0,1.0
1,3,2018-09-01,2018-08,0,1.0
2,3,2018-10-01,2018-08,0,4.0
3,3,2018-11-01,2018-08,0,1.0
4,3,2018-12-01,2018-08,1,3.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,158.0
515295,99998,2020-01-01,2018-07,1,1.0
515296,99998,2020-07-01,2018-07,1,2.0
515297,99998,2021-01-01,2018-07,1,3.0


In [87]:
click_home4.describe()

,customer_id,flagging,click_homepage
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,6.133404
std,28972.061729,0.485361,15.005130
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,1.000000
50%,49619.000000,0.000000,3.000000
75%,74951.000000,1.000000,6.000000
max,99998.000000,1.000000,1875.000000


In [88]:
# menentukan kategori
def func(row):
    if row <= 5:
        return 1
    else:
        return 2

click_home4['click_homepages'] = click_home4['click_homepage'].apply(func)

In [89]:
asess_8 = pd.crosstab(click_home4["click_homepages"], click_home4["flagging"], margins=True, margins_name="Total")
asess_8

flagging,0,1,Total
click_homepages,,,
1,197899,173860,371759
2,121641,21899,143540
Total,319540,195759,515299


In [90]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_8["%nonevent"] = asess_8.apply(nonevent, axis = 1)
asess_8["%event"] = asess_8.apply(event, axis = 1)

In [91]:
#Let's calculate WoE and save it to new a column WoE
asess_8["WoE"] = np.log(asess_8["%event"]/asess_8["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_8["IV"] = asess_8["WoE"]*(asess_8["%event"] - asess_8["%nonevent"])

#Let's sum IV
asess_8["IV"].sum() # strong Predictor

0.42609505665421077

## 9. total transaction 

In [92]:
total_trx = trx2[['created_at','customer_id','session_id']].copy()

total_trx['created_at'] = pd.to_datetime(total_trx["created_at"])
total_trx['created_at'] = total_trx['created_at'].dt.strftime('%Y-%m')
total_trx['created_at'] = pd.to_datetime(total_trx["created_at"])

total_trx2 = total_trx.groupby(['created_at','customer_id']).count().reset_index()
total_trx2 = total_trx2.rename(columns={'session_id' : 'total_transaction'})
total_trx2

,created_at,customer_id,total_transaction
0,2016-06-01,74089,1
1,2016-07-01,221,1
2,2016-07-01,837,1
3,2016-07-01,933,1
4,2016-07-01,1022,1
...,...,...,...
565536,2022-07-01,99936,14
565537,2022-07-01,99976,5
565538,2022-07-01,99986,1
565539,2022-07-01,99988,3


In [93]:
total_trx3 = final_flagging.merge(total_trx2, on=['customer_id','created_at'], how='left')
total_trx3

,customer_id,created_at,first_join_date,flagging,total_transaction
0,3,2018-08-01,2018-08,0,1
1,3,2018-09-01,2018-08,0,1
2,3,2018-10-01,2018-08,0,1
3,3,2018-11-01,2018-08,0,1
4,3,2018-12-01,2018-08,1,1
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,1
515295,99998,2020-01-01,2018-07,1,1
515296,99998,2020-07-01,2018-07,1,2
515297,99998,2021-01-01,2018-07,1,1


In [94]:
total_trx3.describe()

,customer_id,flagging,total_transaction
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,2.143146
std,28972.061729,0.485361,2.400546
min,3.000000,0.000000,1.000000
25%,24742.000000,0.000000,1.000000
50%,49619.000000,0.000000,1.000000
75%,74951.000000,1.000000,2.000000
max,99998.000000,1.000000,78.000000


In [95]:
# menentukan kategori
def func(row):
    if row <= 2:
        return 1
    else:
        return 2

total_trx3['total_transactions'] = total_trx3['total_transaction'].apply(func)

In [96]:
asess_9 = pd.crosstab(total_trx3["total_transactions"], total_trx3["flagging"], margins=True, margins_name="Total")
asess_9

flagging,0,1,Total
total_transactions,,,
1,223530,177512,401042
2,96010,18247,114257
Total,319540,195759,515299


In [97]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_9["%nonevent"] = asess_9.apply(nonevent, axis = 1)
asess_9["%event"] = asess_9.apply(event, axis = 1)

In [98]:
#Let's calculate WoE and save it to new a column WoE
asess_9["WoE"] = np.log(asess_9["%event"]/asess_9["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_9["IV"] = asess_9["WoE"]*(asess_9["%event"] - asess_9["%nonevent"])

#Let's sum IV
asess_9["IV"].sum() # medium Predictor

0.29635825472515

## 10. trx item quantity

In [99]:
trx_item = trx2[['created_at','customer_id','quantity']].copy()

trx_item['created_at'] = pd.to_datetime(trx_item["created_at"])
trx_item['created_at'] = trx_item['created_at'].dt.strftime('%Y-%m')
trx_item['created_at'] = pd.to_datetime(trx_item["created_at"])

trx_item2 = trx_item.groupby(['created_at','customer_id']).sum().reset_index()
trx_item2 = trx_item2.rename(columns={'quantity' : 'item_count'})
trx_item2

,created_at,customer_id,item_count
0,2016-06-01,74089,3
1,2016-07-01,221,1
2,2016-07-01,837,2
3,2016-07-01,933,1
4,2016-07-01,1022,1
...,...,...,...
565536,2022-07-01,99936,16
565537,2022-07-01,99976,5
565538,2022-07-01,99986,1
565539,2022-07-01,99988,3


In [100]:
trx_item3 = final_flagging.merge(trx_item2, on=['customer_id','created_at'], how='left')
trx_item3

,customer_id,created_at,first_join_date,flagging,item_count
0,3,2018-08-01,2018-08,0,5
1,3,2018-09-01,2018-08,0,1
2,3,2018-10-01,2018-08,0,1
3,3,2018-11-01,2018-08,0,1
4,3,2018-12-01,2018-08,1,1
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,1
515295,99998,2020-01-01,2018-07,1,1
515296,99998,2020-07-01,2018-07,1,2
515297,99998,2021-01-01,2018-07,1,1


In [101]:
trx_item3.describe()

,customer_id,flagging,item_count
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,3.162430
std,28972.061729,0.485361,4.244825
min,3.000000,0.000000,1.000000
25%,24742.000000,0.000000,1.000000
50%,49619.000000,0.000000,2.000000
75%,74951.000000,1.000000,3.000000
max,99998.000000,1.000000,129.000000


In [102]:
# menentukan kategori
def func(row):
    if row == 1:
        return 1
    elif row <= 4:
        return 2
    else:
        return 3

trx_item3['items_count'] = trx_item3['item_count'].apply(func)

In [103]:
asess_10 = pd.crosstab(trx_item3["items_count"], trx_item3["flagging"], margins=True, margins_name="Total")
asess_10

flagging,0,1,Total
items_count,,,
1,124213,133235,257448
2,118856,42354,161210
3,76471,20170,96641
Total,319540,195759,515299


In [104]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_10["%nonevent"] = asess_10.apply(nonevent, axis = 1)
asess_10["%event"] = asess_10.apply(event, axis = 1)

In [105]:
#Let's calculate WoE and save it to new a column WoE
asess_10["WoE"] = np.log(asess_10["%event"]/asess_10["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_10["IV"] = asess_10["WoE"]*(asess_10["%event"] - asess_10["%nonevent"])

#Let's sum IV
asess_10["IV"].sum() # strong Predictor

0.36264737541039

## 11. total amount

In [106]:
trx_amount = trx2[['created_at','customer_id','total_amount']].copy()

trx_amount['created_at'] = pd.to_datetime(trx_amount["created_at"])
trx_amount['created_at'] = trx_amount['created_at'].dt.strftime('%Y-%m')
trx_amount['created_at'] = pd.to_datetime(trx_amount["created_at"])

trx_amount2 = trx_amount.groupby(['created_at','customer_id']).sum().reset_index()
trx_amount2

,created_at,customer_id,total_amount
0,2016-06-01,74089,640795
1,2016-07-01,221,467436
2,2016-07-01,837,379716
3,2016-07-01,933,237978
4,2016-07-01,1022,244862
...,...,...,...
565536,2022-07-01,99936,36570250
565537,2022-07-01,99976,3283919
565538,2022-07-01,99986,379377
565539,2022-07-01,99988,2117835


In [107]:
trx_amount3 = final_flagging.merge(trx_amount2, on=['customer_id','created_at'], how='left')
trx_amount3

,customer_id,created_at,first_join_date,flagging,total_amount
0,3,2018-08-01,2018-08,0,826597
1,3,2018-09-01,2018-08,0,163784
2,3,2018-10-01,2018-08,0,124932
3,3,2018-11-01,2018-08,0,247902
4,3,2018-12-01,2018-08,1,181237
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,384029
515295,99998,2020-01-01,2018-07,1,216338
515296,99998,2020-07-01,2018-07,1,1068674
515297,99998,2021-01-01,2018-07,1,366178


In [108]:
trx_amount3.describe()

,customer_id,flagging,total_amount
count,515299.000000,515299.000000,5.152990e+05
mean,49873.821358,0.379894,2.539591e+06
std,28972.061729,0.485361,1.153880e+07
min,3.000000,0.000000,1.520300e+04
25%,24742.000000,0.000000,2.375340e+05
50%,49619.000000,0.000000,4.144470e+05
75%,74951.000000,1.000000,1.166800e+06
max,99998.000000,1.000000,8.931705e+08


In [109]:
# menentukan kategori
def func(row):
    if row <= 240000:
        return 1
    elif row <= 450000:
        return 2
    else:
        return 3

trx_amount3['total_amount_paid'] = trx_amount3['total_amount'].apply(func)

In [110]:
asess_11 = pd.crosstab(trx_amount3["total_amount_paid"], trx_amount3["flagging"], margins=True, margins_name="Total")
asess_11

flagging,0,1,Total
total_amount_paid,,,
1,63855,67498,131353
2,76019,65182,141201
3,179666,63079,242745
Total,319540,195759,515299


In [111]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_11["%nonevent"] = asess_11.apply(nonevent, axis = 1)
asess_11["%event"] = asess_11.apply(event, axis = 1)


In [112]:
#Let's calculate WoE and save it to new a column WoE
asess_11["WoE"] = np.log(asess_11["%event"]/asess_11["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_11["IV"] = asess_11["WoE"]*(asess_11["%event"] - asess_11["%nonevent"])

#Let's sum IV
asess_11["IV"].sum() # medium Predictor

0.24467076085298617

## 12. promo used count

In [113]:
# ketika booking_id sama, dan pake promo yang sama, maka penggunaan promo dihitung 1 (harus groupby booking_id
# dan aggregate promo_amount first)
# baru setelah itu digroupby sum berbulan (created_at)

trx_promo2_first

,customer_id,created_at,booking_id,promo_amount
0,3,2018-08-01,856b0c3d-b076-406b-be11-3dfee762e9be,8073
1,3,2018-09-01,f9b17de8-5db3-40dc-9227-664c6679543c,0
2,3,2018-10-01,caa8be57-6239-4b5b-ae0b-e27887a13356,8828
3,3,2018-11-01,688cef52-8ee7-44ea-98af-20daf0c5609b,3415
4,3,2018-12-01,a12fb778-d19a-4559-9808-8115f01c1595,2645
...,...,...,...,...
815959,99998,2020-07-01,0d102b41-78e1-4c46-81d2-eff6f725fc2a,0
815960,99998,2021-01-01,d7a09532-f774-4528-9d6e-be9ef2d67bef,8569
815961,99998,2021-07-01,0361a084-9362-4a93-b940-d59e7ce91970,0
815962,99998,2022-01-01,06fbe4b4-e98b-403d-a05d-396ae9903e2a,1954


In [114]:
# berapa kali dia pake promo tiap bulannya (diliat berdasarkan ordernya)
promo_used = trx_promo2_first.copy()
promo_used['promo_count'] = promo_used['promo_amount'].apply(lambda x : 1 if x > 0 else 0)
promo_used

,customer_id,created_at,booking_id,promo_amount,promo_count
0,3,2018-08-01,856b0c3d-b076-406b-be11-3dfee762e9be,8073,1
1,3,2018-09-01,f9b17de8-5db3-40dc-9227-664c6679543c,0,0
2,3,2018-10-01,caa8be57-6239-4b5b-ae0b-e27887a13356,8828,1
3,3,2018-11-01,688cef52-8ee7-44ea-98af-20daf0c5609b,3415,1
4,3,2018-12-01,a12fb778-d19a-4559-9808-8115f01c1595,2645,1
...,...,...,...,...,...
815959,99998,2020-07-01,0d102b41-78e1-4c46-81d2-eff6f725fc2a,0,0
815960,99998,2021-01-01,d7a09532-f774-4528-9d6e-be9ef2d67bef,8569,1
815961,99998,2021-07-01,0361a084-9362-4a93-b940-d59e7ce91970,0,0
815962,99998,2022-01-01,06fbe4b4-e98b-403d-a05d-396ae9903e2a,1954,1


In [115]:
promo_used2 = promo_used.groupby(['customer_id','created_at'])[['promo_amount','promo_count']].sum().reset_index()
promo_used2

,customer_id,created_at,promo_amount,promo_count
0,3,2018-08-01,8073,1
1,3,2018-09-01,0,0
2,3,2018-10-01,8828,1
3,3,2018-11-01,3415,1
4,3,2018-12-01,2645,1
...,...,...,...,...
565536,99998,2020-07-01,0,0
565537,99998,2021-01-01,8569,1
565538,99998,2021-07-01,0,0
565539,99998,2022-01-01,1954,1


In [116]:
promo_used3 = final_flagging.merge(promo_used2, on=['customer_id','created_at'], how='left')
promo_used3

,customer_id,created_at,first_join_date,flagging,promo_amount,promo_count
0,3,2018-08-01,2018-08,0,8073,1
1,3,2018-09-01,2018-08,0,0,0
2,3,2018-10-01,2018-08,0,8828,1
3,3,2018-11-01,2018-08,0,3415,1
4,3,2018-12-01,2018-08,1,2645,1
...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,5670,1
515295,99998,2020-01-01,2018-07,1,4912,1
515296,99998,2020-07-01,2018-07,1,0,0
515297,99998,2021-01-01,2018-07,1,8569,1


In [117]:
promo_used3.describe()

,customer_id,flagging,promo_amount,promo_count
count,515299.000000,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,3004.977757,0.563444
std,28972.061729,0.485361,4163.880206,0.685609
min,3.000000,0.000000,0.000000,0.000000
25%,24742.000000,0.000000,0.000000,0.000000
50%,49619.000000,0.000000,0.000000,0.000000
75%,74951.000000,1.000000,5150.000000,1.000000
max,99998.000000,1.000000,70409.000000,14.000000


In [118]:
# menentukan kategori
def func(row):
    if row == 0:
        return 1
    elif row == 1:
        return 2
    else:
        return 3

promo_used3['promo_used'] = promo_used3['promo_count'].apply(func)

In [119]:
asess_12 = pd.crosstab(promo_used3["promo_used"], promo_used3["flagging"], margins=True, margins_name="Total")
asess_12

flagging,0,1,Total
promo_used,,,
1,154835,115058,269893
2,129501,80436,209937
3,35204,265,35469
Total,319540,195759,515299


In [120]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_12["%nonevent"] = asess_12.apply(nonevent, axis = 1)
asess_12["%event"] = asess_12.apply(event, axis = 1)


In [121]:
#Let's calculate WoE and save it to new a column WoE
asess_12["WoE"] = np.log(asess_12["%event"]/asess_12["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_12["IV"] = asess_12["WoE"]*(asess_12["%event"] - asess_12["%nonevent"])

#Let's sum IV
asess_12["IV"].sum() # strong Predictor

0.498709232712254

## 13. click add_to_cart

In [122]:
ses_add = click_new[click_new['event_name'] == 'ADD_TO_CART'][['session_id','event_name']]
add_cart = trx2.merge(ses_add, on='session_id')

In [123]:
add_cart['created_at'] = pd.to_datetime(add_cart["created_at"])
add_cart['created_at'] = add_cart['created_at'].dt.strftime('%Y-%m')
add_cart['created_at'] = pd.to_datetime(add_cart["created_at"])

add_cart2 = add_cart.groupby(['created_at','customer_id'])['event_name'].count().reset_index()
add_cart2 = add_cart2.rename(columns = {'event_name' : 'add_cart_count'})

In [124]:
add_cart3 = final_flagging.merge(add_cart2, on=['created_at','customer_id'], how='left')
add_cart3 = add_cart3.fillna(0)
add_cart3

,customer_id,created_at,first_join_date,flagging,add_cart_count
0,3,2018-08-01,2018-08,0,2.0
1,3,2018-09-01,2018-08,0,3.0
2,3,2018-10-01,2018-08,0,3.0
3,3,2018-11-01,2018-08,0,3.0
4,3,2018-12-01,2018-08,1,4.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,2.0
515295,99998,2020-01-01,2018-07,1,1.0
515296,99998,2020-07-01,2018-07,1,6.0
515297,99998,2021-01-01,2018-07,1,1.0


In [125]:
add_cart3.describe()

,customer_id,flagging,add_cart_count
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,8.458757
std,28972.061729,0.485361,31.226275
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,1.000000
50%,49619.000000,0.000000,2.000000
75%,74951.000000,1.000000,5.000000
max,99998.000000,1.000000,2661.000000


In [126]:
# menentukan kategori
def func(row):
    if row <= 2:
        return 1
    elif row <= 6:
        return 2
    else:
        return 3

add_cart3['cart_count'] = add_cart3['add_cart_count'].apply(func)

In [127]:
asess_12 = pd.crosstab(add_cart3["cart_count"], add_cart3["flagging"], margins=True, margins_name="Total")
asess_12

flagging,0,1,Total
cart_count,,,
1,169647,108757,278404
2,79219,57482,136701
3,70674,29520,100194
Total,319540,195759,515299


In [128]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_12["%nonevent"] = asess_12.apply(nonevent, axis = 1)
asess_12["%event"] = asess_12.apply(event, axis = 1)

In [129]:
#Let's calculate WoE and save it to new a column WoE
asess_12["WoE"] = np.log(asess_12["%event"]/asess_12["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_12["IV"] = asess_12["WoE"]*(asess_12["%event"] - asess_12["%nonevent"])

#Let's sum IV
asess_12["IV"].sum() # weak Predictor

0.0358126783491968

## 13. created_at column

In [130]:
trx_created = trx2[['created_at','customer_id']].copy()

trx_created['created_at'] = pd.to_datetime(trx_created["created_at"])
trx_created['created_at'] = trx_created['created_at'].dt.strftime('%Y-%m')
trx_created['created_at'] = pd.to_datetime(trx_created["created_at"])

trx_created = trx_created[['created_at','customer_id']].drop_duplicates()

In [131]:
trx_created['month'] = trx_created['created_at'].dt.month

def quarter(row):
    if row <= 3:
        return 1
    elif row <= 6:
        return 2
    elif row <= 9:
        return 3
    else:
        return 4

trx_created['quarter'] = trx_created['month'].apply(quarter)
trx_created

,created_at,customer_id,month,quarter
0,2018-07-01,5868,7,3
1,2018-07-01,4774,7,3
2,2018-09-01,4774,9,3
3,2018-11-01,4774,11,4
4,2018-12-01,4774,12,4
...,...,...,...,...
1254579,2020-11-01,25121,11,4
1254581,2021-12-01,34829,12,4
1254582,2021-12-01,99645,12,4
1254583,2019-12-01,87319,12,4


In [132]:
trx_created2 = final_flagging.merge(trx_created.drop(columns=['month']), on=['created_at','customer_id'])
trx_created2

,customer_id,created_at,first_join_date,flagging,quarter
0,3,2018-08-01,2018-08,0,3
1,3,2018-09-01,2018-08,0,3
2,3,2018-10-01,2018-08,0,4
3,3,2018-11-01,2018-08,0,4
4,3,2018-12-01,2018-08,1,4
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,3
515295,99998,2020-01-01,2018-07,1,1
515296,99998,2020-07-01,2018-07,1,3
515297,99998,2021-01-01,2018-07,1,1


In [133]:
asess_13 = pd.crosstab(trx_created2["quarter"], trx_created2["flagging"], margins=True, margins_name="Total")
asess_13

flagging,0,1,Total
quarter,,,
1,83591,52391,135982
2,90565,44450,135015
3,68399,48613,117012
4,76985,50305,127290
Total,319540,195759,515299


In [134]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_13["%nonevent"] = asess_13.apply(nonevent, axis = 1)
asess_13["%event"] = asess_13.apply(event, axis = 1)

In [135]:
#Let's calculate WoE and save it to new a column WoE
asess_13["WoE"] = np.log(asess_13["%event"]/asess_13["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_13["IV"] = asess_13["WoE"]*(asess_13["%event"] - asess_13["%nonevent"])

#Let's sum IV
asess_13["IV"].sum() # useless Predictor

0.01875854405587626

## 14. scroll frequent

In [136]:
click_scroll = click_new[click_new['event_name'] == 'SCROLL'][['session_id']]
click_scroll2 = trx2.iloc[:,[0,1,3]].merge(click_scroll, on='session_id')

click_scroll2['created_at'] = pd.to_datetime(click_scroll2["created_at"])
click_scroll2['created_at'] = click_scroll2['created_at'].dt.strftime('%Y-%m')
click_scroll2['created_at'] = pd.to_datetime(click_scroll2["created_at"])

In [137]:
click_scroll3 = click_scroll2.groupby(['customer_id','created_at']).count().reset_index()
click_scroll3 = click_scroll3.rename(columns={
    'session_id' : 'click_scroll'
})
click_scroll3

,customer_id,created_at,click_scroll
0,3,2018-10-01,4
1,3,2018-12-01,3
2,3,2019-05-01,4
3,3,2019-06-01,8
4,3,2019-07-01,2
...,...,...,...
386317,99998,2020-07-01,6
386318,99998,2021-01-01,1
386319,99998,2021-07-01,1
386320,99998,2022-01-01,48


In [138]:
click_scroll4 = final_flagging.merge(click_scroll3, on=['customer_id','created_at'], how='left')
click_scroll4 = click_scroll4.fillna(0)
click_scroll4

,customer_id,created_at,first_join_date,flagging,click_scroll
0,3,2018-08-01,2018-08,0,0.0
1,3,2018-09-01,2018-08,0,0.0
2,3,2018-10-01,2018-08,0,4.0
3,3,2018-11-01,2018-08,0,0.0
4,3,2018-12-01,2018-08,1,3.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,16.0
515295,99998,2020-01-01,2018-07,1,2.0
515296,99998,2020-07-01,2018-07,1,6.0
515297,99998,2021-01-01,2018-07,1,1.0


In [139]:
click_scroll4.describe()

,customer_id,flagging,click_scroll
count,515299.000000,515299.000000,515299.000000
mean,49873.821358,0.379894,4.168927
std,28972.061729,0.485361,8.971692
min,3.000000,0.000000,0.000000
25%,24742.000000,0.000000,0.000000
50%,49619.000000,0.000000,2.000000
75%,74951.000000,1.000000,4.000000
max,99998.000000,1.000000,480.000000


In [140]:
# menentukan kategori
def func(row):
    if row <= 2:
        return 1
    elif row <= 5:
        return 2
    else:
        return 3

click_scroll4['scroll_count'] = click_scroll4['click_scroll'].apply(func)

In [141]:
asess_14 = pd.crosstab(click_scroll4["scroll_count"], click_scroll4["flagging"], margins=True, margins_name="Total")
asess_14

flagging,0,1,Total
scroll_count,,,
1,162012,155378,317390
2,64949,26021,90970
3,92579,14360,106939
Total,319540,195759,515299


In [142]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_14["%nonevent"] = asess_14.apply(nonevent, axis = 1)
asess_14["%event"] = asess_14.apply(event, axis = 1)

In [143]:
#Let's calculate WoE and save it to new a column WoE
asess_14["WoE"] = np.log(asess_14["%event"]/asess_14["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_14["IV"] = asess_14["WoE"]*(asess_14["%event"] - asess_14["%nonevent"])

#Let's sum IV
asess_14["IV"].sum() # strong Predictor

0.45557876330997776

## 15. search frequent

In [144]:
click_search = click_new[click_new['event_name'] == 'SEARCH'][['session_id']]
click_search2 = trx2.iloc[:,[0,1,3]].merge(click_search, on='session_id')

click_search2['created_at'] = pd.to_datetime(click_search2["created_at"])
click_search2['created_at'] = click_search2['created_at'].dt.strftime('%Y-%m')
click_search2['created_at'] = pd.to_datetime(click_search2["created_at"])

In [145]:
click_search3 = click_search2.groupby(['customer_id','created_at']).count().reset_index()
click_search3 = click_search3.rename(columns={
    'session_id' : 'click_search'
})
click_search3

,customer_id,created_at,click_search
0,3,2018-10-01,1
1,3,2018-11-01,1
2,3,2018-12-01,1
3,3,2019-02-01,1
4,3,2019-03-01,1
...,...,...,...
338072,99998,2019-07-01,9
338073,99998,2020-07-01,2
338074,99998,2021-07-01,1
338075,99998,2022-01-01,16


In [146]:
click_search4 = final_flagging.merge(click_search3, on=['customer_id','created_at'], how='left')
click_search4 = click_search4.fillna(0)
click_search4

,customer_id,created_at,first_join_date,flagging,click_search
0,3,2018-08-01,2018-08,0,0.0
1,3,2018-09-01,2018-08,0,0.0
2,3,2018-10-01,2018-08,0,1.0
3,3,2018-11-01,2018-08,0,1.0
4,3,2018-12-01,2018-08,1,1.0
...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,9.0
515295,99998,2020-01-01,2018-07,1,0.0
515296,99998,2020-07-01,2018-07,1,2.0
515297,99998,2021-01-01,2018-07,1,0.0


In [147]:
click_search4.describe()

,customer_id,flagging,click_search
count,515299.000000,515299.000000,515299.00000
mean,49873.821358,0.379894,2.94502
std,28972.061729,0.485361,6.68540
min,3.000000,0.000000,0.00000
25%,24742.000000,0.000000,0.00000
50%,49619.000000,0.000000,1.00000
75%,74951.000000,1.000000,3.00000
max,99998.000000,1.000000,374.00000


In [148]:
# menentukan kategori
def func(row):
    if row <= 1:
        return 1
    elif row <= 3:
        return 2
    else:
        return 3

click_search4['search_count'] = click_search4['click_search'].apply(func)

In [149]:
asess_15 = pd.crosstab(click_search4["search_count"], click_search4["flagging"], margins=True, margins_name="Total")
asess_15

flagging,0,1,Total
search_count,,,
1,153011,148551,301562
2,65487,30075,95562
3,101042,17133,118175
Total,319540,195759,515299


In [150]:
# Let's create function nonevent & event
def nonevent(df) : 
    return (df[0]/319540)
def event(df) : 
    return (df[1]/195759)

# Apply function nonevent & event and save it to new columns %nonevent & %event
asess_15["%nonevent"] = asess_15.apply(nonevent, axis = 1)
asess_15["%event"] = asess_15.apply(event, axis = 1)

In [151]:
#Let's calculate WoE and save it to new a column WoE
asess_15["WoE"] = np.log(asess_15["%event"]/asess_15["%nonevent"])

# Let's calculate IV and save it to new a column IV
asess_15["IV"] = asess_15["WoE"]*(asess_15["%event"] - asess_15["%nonevent"])

#Let's sum IV
asess_15["IV"].sum() # strong Predictor

0.4374605594295019

# Feature Selection

In [152]:
feature_all = trx_click6.merge(trx_add3.drop(columns=['quantity']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(trx_promo3.drop(columns=['promo_amount']), on=['customer_id','created_at','first_join_date','flagging'])
# feature_all = feature_all.merge(trx_method6, on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(trx_shipment3.drop(columns=['shipment_fee']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(trx_datelimit3.drop(columns=['diff_datelimit']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(click_item4.drop(columns=['click_detail_item']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(click_home4.drop(columns=['click_homepage']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(total_trx3.drop(columns=['total_transaction']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(trx_item3.drop(columns=['item_count']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(trx_amount3.drop(columns=['total_amount']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(promo_used3.drop(columns=['promo_amount','promo_count']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(add_cart3.drop(columns=['add_cart_count']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(click_scroll4.drop(columns=['click_scroll']), on=['customer_id','created_at','first_join_date','flagging'])
feature_all = feature_all.merge(click_search4.drop(columns=['click_search']), on=['customer_id','created_at','first_join_date','flagging'])

feature_all = feature_all.fillna(0)

In [153]:
feature_all

,customer_id,created_at,first_join_date,flagging,mobile_used,web_used,add_to_cart_item,promo_amount_used,shipments_fee,diff_datelimits,click_detail_items,click_homepages,total_transactions,items_count,total_amount_paid,promo_used,cart_count,scroll_count,search_count
0,3,2018-08-01,2018-08,0,1.0,0.0,2,3,1,1,1,1,1,3,3,2,1,1,1
1,3,2018-09-01,2018-08,0,1.0,0.0,2,1,1,2,1,1,1,1,1,1,2,1,1
2,3,2018-10-01,2018-08,0,1.0,0.0,2,3,1,4,2,1,1,1,1,2,2,2,1
3,3,2018-11-01,2018-08,0,1.0,0.0,2,2,1,2,2,1,1,1,2,2,2,1,1
4,3,2018-12-01,2018-08,1,1.0,0.0,2,2,1,2,1,1,1,1,1,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515294,99998,2019-07-01,2018-07,1,1.0,0.0,1,3,1,2,3,2,1,1,2,2,1,3,3
515295,99998,2020-01-01,2018-07,1,1.0,0.0,1,2,1,2,1,1,1,1,1,2,1,1,1
515296,99998,2020-07-01,2018-07,1,1.0,0.0,2,1,2,3,1,1,1,2,3,1,2,3,2
515297,99998,2021-01-01,2018-07,1,1.0,0.0,1,3,1,1,2,1,1,1,2,2,1,1,1


In [154]:
# feature_all.to_csv('final_feature.csv',index=False)

In [155]:
X = feature_all.iloc[:,4:]
y = feature_all[["flagging"]]

In [156]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.3)
print("Jumlah data training: ", X_train.shape)
print("Jumlah data testing: ", X_test.shape)

Jumlah data training:  (360709, 15)
Jumlah data testing:  (154590, 15)


In [157]:
def calc_iv(df, feature, target, pr=False):
    
    lst = []

    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())

    iv = data['IV'].sum()

    return iv, data

In [158]:
iv_df = X_train.copy()
iv_df["target"] = y_train

features = iv_df.columns[:-1].tolist()

iv_list = []
for feature in features:
    iv, data = calc_iv(iv_df, feature, 'target')
    iv_list.append(round(iv,6))

woe_df = pd.DataFrame(np.column_stack([features, iv_list]), 
                      columns=['Feature', 'iv'])
woe_df

,Feature,iv
0,mobile_used,0.022457
1,web_used,0.032109
2,add_to_cart_item,0.030047
3,promo_amount_used,0.070696
4,shipments_fee,0.242014
5,diff_datelimits,0.058132
6,click_detail_items,0.327078
7,click_homepages,0.423928
8,total_transactions,0.294547
9,items_count,0.359284
